In [1]:
from __future__ import print_function
import pandas as pd
from glob import glob
from time import time

In [2]:
DATA_INTERIM = "../data/interim/"
DATA_EXTERNAL = "../data/external/"
DATA_PROCESSED = "../data/processed/"
var = "phastCons46way"

In [3]:
dbsnp = pd.read_csv("../data/external/dbsnp.csv.gz")
humsavar_dbsnp = pd.read_csv("../data/processed/humsavar_gt_dbSNP.txt")

In [4]:
humsavar_dbsnp.columns

Index([u'dbSNP'], dtype='object')

In [5]:
dbsnp = dbsnp[dbsnp.name.isin(humsavar_dbsnp.dbSNP)]

In [6]:
dbsnp.shape

(4916, 5)

In [7]:
dbsnp["dist"] = dbsnp.chromEnd - dbsnp.chromStart

In [8]:
dbsnp_dist1 = dbsnp[dbsnp.dist == 1]

In [9]:
dbsnp_dist1[dbsnp_dist1[["chromStart", "chromEnd"]].duplicated(keep=False)]

,bin,chrom,chromStart,chromEnd,name,dist
2082453,957,chr15,48760293,48760294,rs111401431,1
6616728,957,chrX,48760293,48760294,rs878853145,1


In [ ]:
res = []
for e, chrom in enumerate(dbsnp_dist1.chrom.unique()):
    print("Chrom: ", chrom)
    timeInit = time()
    dbsnp_filtered = dbsnp_dist1[dbsnp_dist1.chrom == chrom]
    try:
        score_table = pd.read_csv(DATA_INTERIM + "parsed_" + var + "/" + chrom + "." + var + ".csv.gz",
                              header=None, names=["chr", "chromStart", var], usecols=["chromStart", var])
        score_table = score_table[score_table.chromStart.isin(dbsnp_filtered.chromStart)]
        res.append(score_table.merge(dbsnp_filtered, on="chromStart")[["name", var]])
    except Exception as e:
        print("Exception: ", e)
        continue
    timeEnd = time()
    print("Time Elapsed: {:.2f}s.".format(timeEnd - timeInit))
res = pd.concat(res)

In [12]:
res.to_csv(DATA_PROCESSED + var + ".csv", index=False)